In [1]:
ApproximatingPolynomial.<r> = PolynomialRing(ZZ)
NumericalConstant = RealField(128)
LogPolynomial.<u> = PolynomialRing(NumericalConstant)

def get_univariate_bound_polynomial(polynomial):
    result = ApproximatingPolynomial(0)
    for m in polynomial.monomials():
        result += abs(polynomial.monomial_coefficient(m))*r^m.degree()
    return result
def get_univariate_multibound_polynomial(polynomials):
    bounds = [get_univariate_bound_polynomial(p) for p in polynomials]
    result = 0
    for k in range(max([p.degree() for p in bounds])):
        result += max([p.monomial_coefficient(r^k) for p in bounds])*r^k
    return result

def get_height_bound_polynomial(ratio):
    numerator = ratio.numerator(); denominator = ratio.denominator()
    if denominator == 1:
        return get_univariate_bound_polynomial(numerator)
    return get_univariate_multibound_polynomial([numerator, denominator])

def get_height_bound_monomial(ratio):
    polynomial = get_height_bound_polynomial(ratio)
    constant = polynomial.leading_coefficient(); degree = polynomial.degree()
    offset = 0
    while True:
        expansion = ApproximatingPolynomial(constant*(r+offset)^degree)
        if all([expansion.monomial_coefficient(r^i) >= polynomial.monomial_coefficient(r^i) for i in range(degree)]):
            break
        offset += 1
    return (constant, degree, offset)

#
# Performs the following procedure:
# 1. Compute the Weierstrass form of the elliptic curve defined by 'cubic' in the variables 'x' and 'y'.
#    Normalize the curve so that its 'a4' and 'a6' constants are integral, then print them.
# 2. Apply the same transformation to the points from the input, then print their formulas.
# 3. Compute upper bounds for the heights of the new points, then print them.
# 4. Compute the discriminant and j-invariant along with height bounds, and print these.
# 5. Compute and print the resulting upper bound for the regulator.
# 6. Print the lower bound for c(E).
#
# This function requires that 'cubic' is a polynomial in 'x' and 'y' whose coefficients are either integers
# or polynomials with integer coefficients.
#
def compute_lower_bound_for_c(cubic, points, torsion_size):
    rank = len(points)
    
    a4, a6 = WeierstrassForm(cubic, variables = [x,y])
    a4 = a4.constant_coefficient(); a6 = a6.constant_coefficient()
    a4 *= 6^4; a6 *= 6^6
    print("a4 = " + str(a4))
    print()
    print("a6 = " + str(a6))
    print()
    A, B, C = WeierstrassForm(cubic, variables = [x,y], transformation=True)
    X = 6^2*A/C^2; Y = 6^3*B/C^3
    
    x_values = [X.substitute(x=P[0], y=P[1]) for P in points]
    for i in range(rank):
        print("x_values[" + str(i) + "] = " + str(x_values[i]))
        print()
    x_height_bounds = [get_height_bound_monomial(p) for p in x_values]
    for i in range(rank):
        bound = x_height_bounds[i]
        print("x_height_bounds[" + str(i) + "] = " + str(bound[0]) + "*(" + str(r+bound[2]) + ")^" + str(bound[1]))
        print()
    
    discriminant = -16*(4*a4^3+27*a6^2)
    print("discriminant = " + str(discriminant))
    print()
    discriminant_height_bound = get_height_bound_monomial(discriminant)
    print("discriminant_height_bound = " + str(discriminant_height_bound))
    print()
    
    j_invariant = 1728*4*a4^3/discriminant
    print("j_invariant = " + str(j_invariant))
    j_height_bound = get_height_bound_monomial(j_invariant)
    print("j_height_bound = " + str(j_height_bound))
    print()
    
    max_offset = max(max([bound[2] for bound in x_height_bounds]), j_height_bound[2], discriminant_height_bound[2])
    
    regulator_polynomial = 1
    for i in range(rank):
        regulator_polynomial *= LogPolynomial((
                (1/12)*log(j_height_bound[0]) +
                (1/12)*log(discriminant_height_bound[0]) +
                (1/2)*log(x_height_bounds[i][0]) + 1.07
            ) + (
                (1/12)*j_height_bound[1] +
                (1/12)*discriminant_height_bound[1] +
                (1/2)*x_height_bounds[i][1]
            )*u
        )
    print("regulator_polynomial = " + str(regulator_polynomial))
    print()
    
    lower_bound_constant = NumericalConstant(torsion_size*pi^(rank/2)/(2^rank*gamma(rank/2+1)))
    
    print("lower bound for c(E): " + str(lower_bound_constant) + "/sqrt(" + str(regulator_polynomial) + ")")
    print("u = log(" + str(r + max_offset) + ")")
    print()
    return

In [2]:
ParametricPolynomial.<t> = PolynomialRing(ZZ, 1)
ParametricCubic.<x, y> = PolynomialRing(ParametricPolynomial.fraction_field())

A =  2*(87671889*t^24 + 854321688*t^23 + 3766024692*t^22 + 9923033928*t^21+ 17428851514*t^20 + 21621621928*t^19 + 19950275060*t^18+ 15200715960*t^17 + 11789354375*t^16 + 10470452464*t^15 + 8925222696*t^14 + 5984900048*t^13 + 2829340620*t^12 + 820299856*t^11 + 59930952*t^10- 66320528*t^9 - 35768977*t^8 - 9381000*t^7 - 1017244*t^6 + 262760*t^5 + 159130*t^4 + 41096*t^3 + 6468*t^2 + 600*t + 25)

B = (t^2 -2*t-1)^2*(69*t^4 +148*t^3 +78*t^2 +4*t+1)^2*(13*t^2 -2*t-1)^2*(9*t^4 +28*t^3 +18*t^2 +4*t+1)^2*(11*t^4 +12*t^3 +2*t^2 -4*t-1)^2*(9*t^2 +14*t+7)^2*(31*t^4 +52*t^3 +22*t^2 -4*t-1)^2*(3*t^2+2*t+1)^2

cubic = ParametricCubic(y^2 - x^3 - A*x^2 - B*x)

x1 = (9*t^4 +28*t^3 +18*t^2 +4*t+1)^2*(11*t^4 +12*t^3 +2*t^2 -4*t-1)^2*(69*t^4 +148*t^3 +78*t^2 +4*t+1)^2
x2 = (3*t^2 +2*t+1)*(9*t^2 +14*t+7)^2*(13*t^2-2*t-1)*(9*t^4 +28*t^3 +18*t^2 +4*t+1)*(11*t^4 +12*t^3 +2*t^2 -4*t-1)^2*(31*t^4 +52*t^3 +22*t^2 -4*t-1)
x3 = (3*t^2 +2*t+1)*(9*t^2 +14*t+7)^2*(13*t^2-2*t-1)*(9*t^4 +28*t^3 +18*t^2 +4*t+1)^2*(11*t^4 +12*t^3 +2*t^2 -4*t-1)*(69*t^4 +148*t^3 +78*t^2 +4*t+1)
x4 = -(3*t^2 +2*t+1)^2*(9*t^2 +14*t+7)^2*(11*t^4 +12*t^3 +2*t^2 -4*t-1)^2*(31*t^4 +52*t^3 +22*t^2 -4*t-1)^2

m = var('m')
y1 = solve(m^2 - x1^3 - A*x1^2 - B*x1, m)[0].rhs()
print("y1 = " + str(y1))
print()
y2 = solve(m^2 - x2^3 - A*x2^2 - B*x2, m)[0].rhs()
print("y2 = " + str(y2))
print()
y3 = solve(m^2 - x3^3 - A*x3^2 - B*x3, m)[0].rhs()
print("y3 = " + str(y3))
print()
y4 = solve(m^2 - x4^3 - A*x4^2 - B*x4, m)[0].rhs()
print("y4 = " + str(y4))
print()

points = [(ParametricPolynomial(x1),ParametricPolynomial(y1)),(ParametricPolynomial(x2),ParametricPolynomial(y2)),(ParametricPolynomial(x3),ParametricPolynomial(y3)),(ParametricPolynomial(x4),ParametricPolynomial(y4))]

compute_lower_bound_for_c(cubic, points, 8)

y1 = -860924250450*t^36 - 14360980435560*t^35 - 111916294479900*t^34 - 543758677891560*t^33 - 1853825937429690*t^32 - 4728975566172480*t^31 - 9386310648784160*t^30 - 14849205421172800*t^29 - 18953204939595880*t^28 - 19504327685137120*t^27 - 15855098157137040*t^26 - 9534155242836960*t^25 - 3281772838340200*t^24 + 736574643887680*t^23 + 2061656405420320*t^22 + 1657402600338240*t^21 + 791752811447300*t^20 + 192724718198800*t^19 - 34294420134440*t^18 - 57972906776560*t^17 - 30002082990220*t^16 - 9580749337280*t^15 - 2009498066400*t^14 - 300561229760*t^13 - 83463135240*t^12 - 53830630880*t^11 - 27295795600*t^10 - 9777458400*t^9 - 2625026120*t^8 - 549602880*t^7 - 91670560*t^6 - 12316480*t^5 - 1334850*t^4 - 114920*t^3 - 7580*t^2 - 360*t - 10

y2 = -1948398321870*t^36 - 29771515753848*t^35 - 212935007680596*t^34 - 949904733007512*t^33 - 2968195467618918*t^32 - 6909200257083840*t^31 - 12424929068206944*t^30 - 17639331827310528*t^29 - 19974345294912536*t^28 - 17995588696709024*t^27 - 12564121642